# Autonomously Adjusting Reference Voltage Tests
The purpose of autonomously adjusting reference voltage (AARV) is to mitigate rapid voltage change (RVC), without affecting the utility’s control of steady-state voltage or the DER’s steady-state reactive power.  AARV may be used with a volt-watt function that mitigates steady-state overvoltage, but otherwise AARV does not regulate steady-state voltage.

## Code Samples

There are Python code cells in this notebook that create plots and text outputs to illustrate various points of application, implementation, and testing. A live version of this notebook is available at [gridhub](https://github.com/temcdrm/gridhub), under an open-source license and copyright that permit reuse and modification. Run the following Python code cell to define support functions using [Numpy](https://numpy.org/) and [Matplotlib](https://matplotlib.org/).

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math

# convert center (Vref), deadband, slope, q limits, and q bias into a table of V and Q points.
# the function returns two arrays for the V and Q points
#    the arrays have sentinel elements below V1 and above V4, so they are 6 elements long (not 4)
#    the sentinel elements clarify that constant extrapolation is used outside the range [V1..V4]
#    this version of the function does not check any limits - the caller must do that
def set_characteristic (Vref=1.0, deadband=0.0, slope=22.0, qmax=0.44, qmin=-0.44, qbias=0.0):
  Q1 = qmax
  Q2 = qbias
  Q3 = qbias
  Q4 = qmin
  V2 = Vref - 0.5 * deadband
  V3 = Vref + 0.5 * deadband
  V1 = V2 - (Q1 - Q2) / slope
  V4 = V3 - (Q4 - Q3) / slope
  VL = V1 - 0.01
  VH = V4 + 0.01
  vtable = np.array ([VL, V1, V2, V3, V4, VH])
  qtable = np.array ([Q1, Q1, Q2, Q3, Q4, Q4])
  return vtable, qtable

# This function plots a volt-var characteristic, and returns the VQ points. 
# (Tabular display is not needed in this notebook.)
def show_characteristic (label, Vref, deadband, slope, qmax, qmin, qbias=0):
  vtable, qtable = set_characteristic (Vref, deadband, slope, qmax, qmin, qbias)
  
  # bounds for plotting the horizontal axis
  vmin = vtable[0]-0.01
  vmax = vtable[-1]+0.01
  
  # evaluate the characteristic over 500 equal voltage intervals
  v = np.linspace (vmin, vmax, 501)
  # interpolating Q using the numpy library function
  q = np.interp (v, vtable, qtable)
  
  # create the plot
  fig, ax = plt.subplots(1, 1, figsize=(6,4), tight_layout=True)
  fig.suptitle ('{:s} volt-var characteristic'.format (label))
  ax.plot (vtable, qtable, marker='o', color='blue', label='Points and Sentinels')
  ax.plot (v, q, color='red', label='Interpolated')
  ax.grid ()
  ax.set_xlabel ('V [pu]')
  ax.set_ylabel ('Q [pu]')
  ax.set_xlim (vmin, vmax)
  ax.legend ()
  
  plt.show ()
  return vtable, qtable

# This function calculates V and dV at the POC from the formula
# in clause 5.1.2 of IEEE 1547.2-2023
# All input and output in per-unit
def get_voltage (rpu, xpu, dP, dQ, vsrcpu):
  a1 = vsrcpu*vsrcpu + rpu*dP + xpu*dQ
  a2 = xpu*dP - rpu*dQ
  d = math.sqrt (a1*a1 + a2*a2) / vsrcpu / vsrcpu - 1.0
  vpu = vsrcpu + d
  return vpu, d

# use the current directory as default location for the "save plot" buttons
# optimize the graphic export for LaTex and PDF
plt.rcParams['savefig.directory'] = os.getcwd()
plt.rcParams['savefig.pad_inches'] = 0.05
plt.rcParams['savefig.dpi'] = 300.0
plt.rcParams['savefig.bbox'] = 'tight'
# invoke the Jupyter support for  Matplotlib graphics
%matplotlib widget

## AARV Test from 1547.1-2020

Clause 5.14.5 of IEEE 1547.1-2020 describes a specific test for AARV. In all other tests, AARV is to be disabled. Four AARV tests are described, at the time constant values *Tref* = 300s and 5000s, each of those with source voltage steps to (V1+V2)/2 and (V3+V4)/2. Before each voltage step, the steady-state reactive power, Q, should be zero at V=Vref. The volt-var characteristic V1..V4 and Q1..Q4 points are set to the default values.

The voltage steps result in step changes of Q to 0.5\*Q1 and 0.5\*Q4, respectively, with some delay from the open-loop response. These new Q values should follow an exponential decay back to zero, governed by *Tref* = 300s or 5000s. The test criteria specifies that after one time constant, *Tref*, the value of Q should be less than 10% of Q1 or Q4, respectively. However, this behavior depends on whether the volt-var characteristic includes a deadband.

*Tref* is defined as the time constant for AARV reference voltage. On the other hand, *Tresponse* is defined as an open-loop response time, which is converted to an exponential time constant in the Python code below. Furthermore, the simulation is run at a constant time step, *dt*, so both time constants may be implemented as constant decrement factors. If the real hardware controller operates at a constant sample interval, it may also use constant decrement factors to save the time of repeatedly evaluating the exponential functions.

Passing this test verifies implementation of *Tref*, but not the impact of AARV on a system with grid impedance.

### Simulation of AARV Tests

Run the following Python code cell to define a function that simulates and plots an AARV test. The input parameters are:

- *tag* is a text label to appear in the plot.
- *Vref*, *dB*, *K*, *Qmax*, and *Qmin* are the system parameters to define the volt-var characteristic
- *Tref* is the AARV time constant, which ranges from 300s to 5000s. No default value was provided in IEEE 1547, but 300s works well whenever AARV is enabled.
- *Tresponse* is the open-loop response time, which ranges from 1s to 90s. The default in IEEE 1547 is 10s for category A and 5s for category B.
- *dt* is the time step for simulation and plotting.
- *bShiftTable* is a flag to choose between two different implementations of AARV:
    - *True* means that the V1..V4 points in the volt-var characteristic are shifted each time *VRef* changes. This procedure strictly follows language in the footnote in IEEE 1547-2018, and it performs as intended in simulation. However, it's not efficient and it may lose track of the original settings for V1..V4.
    - *False* means that instead of modifying V1..V4, the voltage used for interpolation is adjusted with an offset to the **original** value of *Vref*, denoted as *Vset* in the code. This method also performs as intended, without regenerating the interpolation table at each time step, and without losing track of the original settings for V1..V4. In the Python function signature, the default value is *False* to indicate that this might be the preferred implementation. (It is also close to the way *OpenDSS* implements AARV).
 
The function runs for one time constant, *Tref*, and it provides numerical outputs of the peak and final Q. The test procedure ensures that limits on *Vref* will be met, so the function does not check the limits during execution.

In [ ]:
def show_aarv_test (tag, Vref, dB, K, Qmax, Qmin, Tref, Tresponse, dt, bShiftTable=False):
  TauOL = Tresponse / 2.3026
  IncRef = 1.0 - math.exp(-dt/Tref)
  IncOL = 1.0 - math.exp(-dt/TauOL)
  vtable, qtable = set_characteristic (Vref, dB, K, Qmax, Qmin)

  vtest = [(vtable[1]+vtable[2])/2.0, (vtable[3]+vtable[4])/2.0]
  qtest = np.interp (vtest, vtable, qtable)

  tmax = 1.0 * Tref
  t = np.linspace (0, tmax, int(tmax / dt) + 1)
  npts = len(t)
  v0 = (vtable[1]+vtable[2])/2.0
  q0 = np.interp(v0, vtable, qtable)
  q = np.zeros(npts)
  qpeak = 0.0
  # Keep track of the initial Vref setting for 1) bShiftTable=False, and 2) plotting the original volt-var characteristic
  Vset = Vref 
  # print ('       t      Vpu     Vref     Verr    Qtarg        Q')
  if bShiftTable: # this approach resets the V1..V4 points each time Vref changes
    for i in range(1, npts):
      # apply the reference voltage time constant
      verr = v0 - Vref
      Vref = Vref + verr * IncRef
      #######################################################
      # calculate Q with shifted interpolation table
      vtable, qtable = set_characteristic (Vref=Vref, deadband=dB, slope=K, qmax=Qmax, qmin=Qmin)
      qtarg = np.interp (v0, vtable, qtable)
      #######################################################
      # apply the open-loop response
      q[i] = q[i-1] + (qtarg - q[i-1]) * IncOL
      if abs(q[i]) > qpeak:
        qpeak = abs(q[i])
      # print ('{:8.3f} {:8.5f} {:8.5f} {:8.5f} {:8.5f} {:8.5f}'.format (t[i], v0, Vref, verr, qtarg, q[i])) 
  else: # this approach modifies the voltage entry point
    for i in range(1, npts):
      # apply the reference voltage time constant
      verr = v0 - Vref
      Vref = Vref + verr * IncRef
      #######################################################
      # calculate Q with offset voltage entry point
      qtarg = np.interp (v0 + (Vset - Vref), vtable, qtable)
      #######################################################
      # apply the open-loop response
      q[i] = q[i-1] + (qtarg - q[i-1]) * IncOL
      if abs(q[i]) > qpeak:
        qpeak = abs(q[i])
    #print ('{:8.3f} {:8.5f} {:8.5f} {:8.5f} {:8.5f} {:8.5f}'.format (t[i], v0, vref, verr, qtarg, q[i])) 

#  print ('{:d} test points, IncRef={:.6f}, IncOL={:.6f}, Qpeak={:.3f}, Qend={:.3f}'.format (npts, IncRef, IncOL, qpeak, q[-1]))

  fig, ax = plt.subplots(1, 2, sharex = 'col', figsize=(10,4), constrained_layout=True)
  fig.suptitle ('1547.1-2020 AARV test, {:s}, Tref={:.1f}, Tresponse={:.1f}, Voltage Step to Vs={:.3f}'.format (tag, Tref, Tresponse, v0))

  ax[0].set_title ('Characteristic, K={:.2f}, dB={:.2f}, Qmax={:.2f}'.format (K, dB, Qmax))
  # remove any shift of vtable, qtable that might have been done while simulating the test
  vtable, qtable = set_characteristic (Vset, dB, K, Qmax, Qmin)
  ax[0].plot (vtable, qtable, marker='o', color='blue', label='Points and Sentinels')
  ax[0].plot (vtest, qtest, color='red', marker='s', linestyle='None', label='Test Conditions')
  ax[0].set_xlabel ('V [pu]')
  ax[0].set_xlim (vtable[0]-0.01, vtable[-1]+0.01)
  ax[0].legend()

  ax[1].set_title ('Peak Q={:.4f}, Final Q={:.4f} pu'.format (qpeak, q[-1]))
  ax[1].plot (t, q, color='red', label='Dynamic')
  ax[1].plot (t, q0*np.ones(npts), color='blue', label='Qs={:.3f}'.format(q0))
  ax[1].set_xlabel ('t [s]')
  ax[1].set_xlim (t[0], t[-1])

  for i in range(2):
    ax[1].legend()
    ax[i].grid()
    ax[i].set_ylabel ('Q [pu]')

  plt.show()

### Examples of AARV Tests

Run the following Python code cell to simulate three example AARV tests. In each test, the voltage steps to (V1+V2)/2. The volt-var characteristic is plotted to the left, and the dynamic response of Q is plotted to the right.

- For default Category A, there is no deadband. The steady-state value of Qs would be 0.125 pu, but Q only reaches 0.1174 pu due to the open-loop response time constant. The value of Q at *Tref* is 0.0467 pu, which is 18.68% of Q1, so this result does not pass the test.
- For default Category B, there is a deadband. The steady-state value of Qs would be 0.22 pu, but Q only reaches 0.2083 pu due to the open-loop response time constant. The value of Q at *Tref* is 0.0, because the voltage has entered the deadband by that time, so this result passes the test.
- For Catagory B with no deadband and maximum slope (or gain), Q reaches 0.2122, considering the open-loop response time constant along with the higher gain. The final value of Q at *Tref* is 0.0815 pu, which is 18.52%, so this result does not pass the test.


In [ ]:
show_aarv_test ('cat A', Vref=1, dB=0.0, K=2.5, Qmax=0.25, Qmin=-0.25, Tref=300, Tresponse=10, dt=0.1, bShiftTable=True)  # for default category A
show_aarv_test ('cat B', Vref=1, dB=0.04, K=22.0/3.0, Qmax=0.44, Qmin=-0.44, Tref=300, Tresponse=5, dt=0.1, bShiftTable=True)  # for default category B
show_aarv_test ('cat B', Vref=1, dB=0.0, K=22.0, Qmax=0.44, Qmin=-0.44, Tref=300, Tresponse=5, dt=0.1, bShiftTable=True)  # for steepest, no deadband category B

### Conclusion

The test in clause 5.14.5 of IEEE 1547.1-2020 is only passed for the default Category B characteristic. Q decreases to 0 at t=Tref because V is within the deadband by that time, and 0 is less than 10% of Q1. However, with a deadband, the test fails because Q decreases to about 19% of Q1 at t=Tref. This occurs for the default Category A characteristic, and for a higher-gain Category B characteristic with no deadband. Subgroup 7 should review the test specification for correctness.

AARV should be used without a deadband, so the Tref test procedure should focus on that use case. All three numbered volt-var characteristics in IEEE 1547.1-2020, in Tables 26-28, include a deadband. There should at least be a test characteristic for Category B without a deadband.

## BSD 3-Clause License

Copyright (c) 2023-2024, Meltran, Inc

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.